In [231]:
!pip install azureml
!pip install azureml.core
!pip install sentence_transformers
!pip install elasticsearch

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
  Using cached azureml_core-1.43.0-py3-none-any.whl (2.7 MB)
ERROR: azureml-train-automl-runtime 1.42.0 has requirement azureml-automl-core~=1.42.0, but you'll have azureml-automl-core 1.43.0 which is incompatible.
ERROR: azureml-train-automl-runtime 1.42.0 has requirement azureml-core~=1.42.0, but you'll have azureml-core 1.43.0 which is incompatibl

In [232]:
import os
from sentence_transformers import SentenceTransformer, models, CrossEncoder
from azureml.core import Workspace
from azureml.core.model import Model
from elasticsearch import Elasticsearch, helpers

In [234]:
ws = Workspace.from_config()
keyvault = ws.get_default_keyvault()
es_host = keyvault.get_secret(name='AZ-SD-ES-HOST')
es_user = keyvault.get_secret(name='AZ-SD-ES-USER')
es_pwd = keyvault.get_secret(name='AZ-SD-ES-PWD')
es_conn = Elasticsearch([es_host], basic_auth=(es_user, es_pwd), request_timeout=60)

In [235]:
neural_model_path = Model.get_model_path("c19_ance_msmarco_passage", 1, ws)
neural_model_embedding = SentenceTransformer(neural_model_path)
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-12-v2')

In [236]:
query_search_size = 10
fields = ['hash_id',
           'title',
           'publish_year',
           'journal',
           'topic_id',
           'doi',
           'pubmed_id',
           'text',
           'is_coronavirus',
           'is_coronavirus_title',
           'is_sars_cov2',
           'is_sars_cov2_title',
           'is_sars_cov',
           'is_sars_cov_title',
           'is_mers',
           'is_mers_title',
           'author_count',
           'paper_citation_count',
           'paper_pagerank',
           'score_mf1',
           'score_mf2',
           'score_mf3',
           'score_mf4',
           'text_processed_vector']

In [252]:
query_search_phrase = ["What T-cell epitopes have been identified in the Receptor Binding Motif (RBM) region of the S-glycoprotein Receptor Binding Domain (RBD) of the SARS-CoV-2 virus?"]
#query_search_phrase = ["Which of the current vaccines in the clinic have reported the highest levels of neutralizing Abs after a single vaccination?"]
search_type = 'qv'
q_year_from = 2020
q_year_to = 2021
terms = {}

query_embedding = neural_model_embedding.encode(query_search_phrase)
query_embedding = query_embedding.tolist()

if search_type == 'q':
    match = {
                'multi_match': {
                    'query': query_search_phrase[0],
                    'fields': ['title',
                                'text']
                }
            }
else:
    match = {
                'script_score': {
                    'query': {
                        'match_all': {}
                    },
                    'script': {
                        'source': 'cosineSimilarity(params.queryVector, "text_processed_vector") + 1.0',
                        'params': {
                            'queryVector': query_embedding[0]
                        }
                    }
                }
            }

body = {
        'query': {
            'bool': {
                'must': [
                    match,
                    {
                        'terms': terms
                    },
                    {
                        'range': {
                            'publish_year': {
                                'gte': q_year_from,
                                'lte':q_year_to
                            }
                        }
                    }
                ]
            }
        },
        'fields': fields,
        '_source': False
    }

if not terms:
        del body['query']['bool']['must'][1]

In [255]:
results = es_conn.search(index='pub_text', size=query_search_size, body=body)

for hit in results['hits']['hits'][0:query_search_size]:
    print("\t{}\t{}\t{}".format(hit['_score'], hit['fields']['title'][0][:80], hit['fields']['text'][0][:160]))


	60.784584	Development of a serological assay to identify SARSCoV2 antibodies in COVID19 pa	The presence of antibodies to SARS-CoV-2 indicates the potential of protective immunity [19] . The target antigen of the antibody response to SARS-CoV-2 is a la
	29.162775	Development of a serological assay to identify SARSCoV2 antibodies in COVID19 pa	The copyright holder for this this version posted September 13, 2020. . https://doi.org/10.1101/2020.09.11.20192690 doi: medRxiv preprint EUROIMMUN assays) [36,
	28.620134	Development of a serological assay to identify SARSCoV2 antibodies in COVID19 pa	To ensure the detectability of antibody binding in the sera is specific to the antigen and not non- (Table 2) for IgG antibodies to the S protein. In one COVID-
	25.552803	Molecular features similarities between SARSCoV2 SARS MERS and key human genes c	Moreover, our analyses allowed us to distinguish not only the main factors that 501 contribute to the distribution of the genes along the axes in PCA